In [1]:
import sys
from bs4 import BeautifulSoup
import scrapy
from lxml.etree import fromstring
import requests
import time
import re
import json
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from six.moves.urllib import parse
import selenium.webdriver.chrome.service as service
from pyvirtualdisplay import Display
from selenium.webdriver.support.ui import WebDriverWait

# scroll page function opens the webpage and keep scrollinig till the end of the page until no more new results are being fetched

In [2]:
def scroll_page(url):
    driver = webdriver.Chrome('/Users/mohan/Documents/medius_health_scraper/chromedriver')
    driver.get(url)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)
    html = driver.find_element_by_class_name('css-y1gt6f').get_attribute('innerHTML')
    return html
def get_soup(x):
    soup = BeautifulSoup(x,'lxml')
    return soup

# extract_name fun returns the name of the disease in the webpage
# extract_url fun returns the url or link for each disease page

In [3]:
def extract_name(x):
    name = x.find('h2',{'class':'h1 css-kjyn3a egb32mv1'})
    name = name.text
    return name
def extract_url(x):
    url = x.find('a',attrs = {'href':re.compile('/health/*')})
    url = url.get('href')
    complete_url = 'https://www.healthline.com'+url
    return complete_url

# extract_details fun returns the symptoms listed for each disease in the web page

In [14]:
def extract_details(x):
    name = extract_name(x)
    complete_url = extract_url(x)
    response = requests.get(complete_url)
    source_code = response.text
    sub_soup = get_soup(source_code)
    symptoms = sub_soup.find('a',attrs = {'name':re.compile('symptoms*')})
    return name,complete_url,symptoms

In [5]:
start_url = 'https://www.healthline.com/symptom/dizziness'

In [6]:
def extract_symptoms(symptoms):
    symptoms_list = []
    counter = symptoms.findNext('ul').contents
    for i in range(len(counter)):
        if counter[i] != ' ':
            symptoms_list.append(symptoms.findNext('ul').contents[i].string)
    symptoms_list = list(filter(None.__ne__, symptoms_list))
    return symptoms_list

# start_scraping fun recursively parse through all the pages of all diseases and get symptoms associated with each disease
# final_symptoms is a dictionary with key as disease and values being the symptoms associated with each disease

In [7]:
final_symptoms = {}
def start_scraping(soup):
    for disease in soup.findAll('li', {'class': 'css-ee6cf6'}):
        name,complete_url,symptoms = extract_details(disease)
        print(name)
        print(complete_url)
        if symptoms:
            symptoms_list = []
            symptoms_list = extract_symptoms(symptoms)
            final_symptoms[name] = symptoms_list
            #pass
            print(symptoms_list)
        else:
            #sub_html = scroll_page(complete_url)
            response = requests.get(complete_url)
            source_code = response.text
            sub_soup = get_soup(source_code)
            start_scraping(sub_soup)


# i have extracted symptoms for about 30 records by iterating recursively through the web pages for demonstration and saved in the sample_output.json file

In [9]:
homepage_html = scroll_page(start_url)
soup = get_soup(homepage_html)
start_scraping(soup)

Low Blood Sugar (Hypoglycemia)
https://www.healthline.com/health/hypoglycemia
['blurry vision', 'rapid heartbeat', 'sudden mood changes', 'sudden nervousness', 'pale skin', 'headache', 'hunger', 'shaking', 'dizziness', 'sweating', 'difficulty sleeping', 'skin tingling']
Labyrinthitis
https://www.healthline.com/health/labyrinthitis
['dizziness', 'vertigo', 'loss of balance', 'nausea and vomiting', 'loss of hearing in the high-frequency range in one ear', 'difficulty focusing your eyes']
What Is Hypotension?
https://www.healthline.com/health/hypotension
Orthostatic Hypotension (Postural Hyoptension)
https://www.healthline.com/health/orthostatic-hypotension
Dehydration
https://www.healthline.com/health/dehydration
Fever
https://www.healthline.com/health/fever
Common Cold Overview
https://www.healthline.com/health/common-cold
['adults over age 65', 'women who are pregnant', 'children under age 5', 'people with weakened immune systems due to chronic illness']
Cold and Flu
https://www.health

KeyboardInterrupt: 

In [15]:
len(final_symptoms)

29

In [10]:
json = json.dumps(final_symptoms)

In [16]:
with open('sample_output.json','w') as output_file:
    output_file.write(json)

# Since the webpage response is slow we extract all the records into our database as json documents and iterate over json documents to get the desired results as we need by parsing through the values/symptoms or combinations as we need.

In [12]:
for key,values in final_symptoms.items():
    if 'dizziness' in values:
        print(key)

Low Blood Sugar (Hypoglycemia)
Labyrinthitis
Caffeine Overdose: How Much Is Too Much?
Heat Emergencies


# A sample interaction which returns a disease having both dizziness and rapid heartbeat as symptoms

In [13]:
for key,values in final_symptoms.items():
    if 'dizziness' and 'rapid heartbeat' in values:
        print(key)

Low Blood Sugar (Hypoglycemia)


# A sample scrap showing list of all 84 possible conditions causing dizziness 

In [17]:
for disease in soup.findAll('li', {'class': 'css-ee6cf6'}):
        name,complete_url,symptoms = extract_details(disease)
        print(name)
        print(complete_url)

Low Blood Sugar (Hypoglycemia)
https://www.healthline.com/health/hypoglycemia
Labyrinthitis
https://www.healthline.com/health/labyrinthitis
What Is Hypotension?
https://www.healthline.com/health/hypotension
Meniere's Disease
https://www.healthline.com/health/menieres-disease
Everything You Need to Know About High Blood Pressure (Hypertension)
https://www.healthline.com/health/high-blood-pressure-hypertension
What Causes Abnormal Heart Rhythms?
https://www.healthline.com/health/abnormal-heart-rhythms
Everything You Need to Know About Stroke
https://www.healthline.com/health/stroke
Heart Attack
https://www.healthline.com/health/heart-attack
Bleeding
https://www.healthline.com/health/bleeding
What Is Hyperventilation?
https://www.healthline.com/health/hyperventilation
Dehydration
https://www.healthline.com/health/dehydration
Panic Disorder
https://www.healthline.com/health/panic-disorder
Motion Sickness
https://www.healthline.com/health/motion-sickness
Heat Emergencies
https://www.healthl